# Makemore.

This is using the names dataset from the karpathy "makemore" repo.  The goal is to create a model that 
generates a sequence (in this case a name) that is similar to the names in the dataset.  I'm using this
as a sanity check that the model can learn, and that the growing operations work (and that the learning rate
schedule for different parts of the model at least naively works).

I'm breaking a lot of "rules" here:  no test set or validation set (I honestly don't really care about performance as long
as the results look roughly correct), no performance metrics of any kind (again, just roughly correct), Tokenization is
a very hacky "bytes" cast, etc.

This is essentially a notebook version of an integration test.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn.functional as F
import building_babel.model as bbm
from building_babel.tokenizers import codepoint_tokenize, codepoint_decode
from tqdm import tqdm

In [3]:
start_token = 0
end_token = 1

In [4]:
class NamesDataset(torch.utils.data.Dataset):
    def __init__(self, filename):
        self.tokenized_names = []
        max_len = 0
        with open(filename,'r') as f:
            for l in f:
                t = torch.tensor([start_token] + codepoint_tokenize(l.strip()) + [end_token], dtype=torch.int64)
                self.tokenized_names.append(t)
                max_len = max(max_len, len(t))

        self.max_len = max_len

    def __getitem__(self, i):
        pad = (0,self.max_len - len(self.tokenized_names[i]))
        return F.pad(self.tokenized_names[i], pad, "constant", 0) # we pad with 0s, but it really doesn't matter, because we have a stop token...

    def __len__(self):
        return len(self.tokenized_names)

In [5]:
def sample(x):
    return torch.multinomial(x, 1)

In [12]:
def generate(t, deterministic=False):
    seq = torch.tensor([[start_token]])
    for i in range(18):
        if deterministic:
            next_token = t(seq)[:,-1].softmax(dim=-1).argmax(dim=-1).view(-1,1)
        else:
            next_token = sample(t(seq)[:,-1].softmax(dim=-1)).view(-1,1)
        if next_token[0,-1] < 2:
            break
        seq = torch.concat([seq, next_token], dim=-1)
    print(codepoint_decode(seq[:,1:].tolist()))

In [7]:
nds = NamesDataset("../data/names.txt")

In [8]:
c = bbm.TransformerConfig(128, 1, 256, head_dim=32)
t = bbm.Transformer(c)


In [9]:
optim = torch.optim.Adam(t.parameters(), lr=3e-5)

In [10]:
dl = torch.utils.data.DataLoader(nds, batch_size=100, shuffle=True)

In [13]:
for i in range(20):
    print(i)
    for b in tqdm(dl):
        #print(b.shape)
        optim.zero_grad()
        out = t(b[...,:-1])
        #print(out.shape)
        loss = F.cross_entropy(out.transpose(1,2), b[...,1:])
        
        loss.backward()
        optim.step()
    with torch.no_grad():
        generate(t, deterministic=True)

0


100%|████████████████████████████████████████████████████████████████████████████████████████| 321/321 [00:05<00:00, 59.43it/s]


['']
1


100%|████████████████████████████████████████████████████████████████████████████████████████| 321/321 [00:05<00:00, 59.51it/s]


['an']
2


...

['arisha']
18


100%|████████████████████████████████████████████████████████████████████████████████████████| 321/321 [00:05<00:00, 58.12it/s]


['alina']
19


100%|████████████████████████████████████████████████████████████████████████████████████████| 321/321 [00:05<00:00, 58.34it/s]

['arisha']


In [14]:
for i in range(10):
    generate(t)

['jiki']
['leendon']
['lekah']
['carian']
['ckaner']
['daran']
['adilyne']


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x87 in position 3: invalid start byte

In [15]:
t.grow(256)
optim.add_param_group({"params": 

In [23]:
for i in range(10):
    generate(t)

['maimah']
['wadim']
['lasya']
['malkone']
['asolei']
['awan']
['grrahin']
['kurix']
['jachir']
['mpoaen']


In [22]:
for i in range(20):
    print(i)
    for b in tqdm(dl):
        #print(b.shape)
        optim.zero_grad()
        out = t(b[...,:-1])
        #print(out.shape)
        loss = F.cross_entropy(out.transpose(1,2), b[...,1:])
        
        loss.backward()
        optim.step()
    with torch.no_grad():
        generate(t, deterministic=True)

0


100%|████████████████████████████████████████████████████████████████████████████████████████| 321/321 [00:23<00:00, 13.51it/s]


['arisha']
1


 29%|█████████████████████████▊                                                               | 93/321 [00:07<00:17, 13.20it/s]


KeyboardInterrupt: 